# Final Project - Segmentation of Honeybees

This project is about segmentation of bee bodies based object detection method.

## 1. Import necessary libraries

In [4]:
import argparse
import logging
import os
from functools import partial
import shutil

import tensorflow as tf
import numpy as np
import cv2

from segmentation import model,dataset, bee_dataset, training_config
from segmentation.model import build_model
from segmentation.results_analysis import find_positions, compute_error_metrics
from segmentation.results_visualization import plot_positions, plot_segmentation_map, plot_TP_FN_FP

## 2. Setting the paramaters

In [5]:
dataset_root_dir = '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/dataset'
num_classes = 3
data_format = 'channels_last'
bg_fg_weight = 0.9
validation_num_files = 10
batch_size= 8
num_steps = 1000
checkpoint_dir = '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints'

## 3. Train the model

Now we need to train our model. Here we are using the tensorlow estimator for that purpose. 

The Estimator object wraps a model which is specified by a model_fn, which, given inputs and a number of other parameters, returns the ops necessary to perform training, evaluation, or predictions.

All outputs (checkpoints, event files, etc.) are written to model_dir, or a subdirectory thereof. If model_dir is not set, a temporary directory is used.

The config argument can be passed tf.estimator.RunConfig object containing information about the execution environment. It is passed on to the model_fn, if the model_fn has a parameter named "config" (and input functions in the same manner). 

In the case of this project the model_fn used is that of the ever reliable Unet for segmentation. This has been handled in the helper python files "unet.py" and "model.py" present in the "segmentation" folder of the repository.  

The dataset is prepared for training and evaluation and then the train_and_evaluate method is called. 

In [7]:
images_root_dir = os.path.join(dataset_root_dir, 'frames') # path to images
labels_root_dir = os.path.join(dataset_root_dir, 'frames_txt') # path to labels

if not (os.path.exists(images_root_dir) and os.path.exists(labels_root_dir)):
    raise FileNotFoundError()

config = training_config.get(dataset_root_dir)

if config is None:
    config = training_config.create(dataset_root_dir, validation_num_files)

estimator = tf.estimator.Estimator(model_fn=partial(build_model,
                                                        num_classes=num_classes,
                                                        data_format=data_format,
                                                        bg_fg_weight=bg_fg_weight),
                                       model_dir=checkpoint_dir,
                                       config=tf.estimator.RunConfig(save_checkpoints_steps=100,
                                                                     save_summary_steps=100))

train_spec = tf.estimator.TrainSpec(input_fn=partial(dataset.make_dataset,
                                                         data_generator=partial(bee_dataset.generate_training,
                                                                                frames_root_dir=images_root_dir,
                                                                                labels_root_dir=labels_root_dir,
                                                                                filenames=config['train']),
                                                         data_format=data_format,
                                                         batch_size=batch_size,
                                                         mode=tf.estimator.ModeKeys.TRAIN), max_steps=num_steps)

eval_spec = tf.estimator.EvalSpec(input_fn=partial(dataset.make_dataset,
                                                       data_generator=partial(bee_dataset.generate_training,
                                                                              frames_root_dir=images_root_dir,
                                                                              labels_root_dir=labels_root_dir,
                                                                              filenames=config['test']),
                                                       data_format=data_format,
                                                       batch_size=batch_size,
                                                       mode=tf.estimator.ModeKeys.EVAL), steps=None)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:segmentation.training_config:Loading training config from /home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/dataset/training_config.json


INFO:tensorflow:Using config: {'_model_dir': '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01483f4250>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01483f4250>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints/model.ckpt-1000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints/model.ckpt-1000


INFO:tensorflow:Loss for final step: 0.09478972.


INFO:tensorflow:Loss for final step: 0.09478972.


({'loss': 0.08129069, 'global_step': 1000}, [])

## 4. Evaluation on test data

Below a function is defined to create a text file and save the file with the position error, correct classes, True Positives, False Negatives and False Positives.

In [8]:
def save_error_file(point_d, axis_d, tps, fps, fns, correct_type, out_file):
    total = len(point_d) + fns
    tps_mn = float(tps) / total
    fns_mn = float(fns) / total
    fps_mn = float(fps) / total
    correct_type_pr = float(correct_type) / tps

    with open(out_file, "w") as f:
        f.write("position error (pixels): mean: {:.2f} ({:.2f}) median: {:.2f}\n".format(np.mean(point_d),
                                                                                         np.std(point_d),
                                                                                         np.median(point_d)))
        f.write("correct class: {:.2f}%\n".format(correct_type_pr * 100))
        axis_d = np.rad2deg(axis_d)
        f.write("axis error (degrees) : mean: {:.2f} ({:.2f}) median {:.2f}\n".format(np.mean(axis_d),
                                                                                      np.std(axis_d),
                                                                                      np.median(axis_d)))
        f.write("True Positives: {:.2f}%\n".format(tps_mn * 100))
        f.write("False Negatives: {:.2f}%\n".format(fns_mn * 100))
        f.write("False Positives: {:.2f}%\n".format(fps_mn * 100))


In [9]:
results_folder = '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/predict_result'
min_distance_px = 20
min_blob_size_px = 20
max_blob_size_px = 200

In [10]:
output_path = os.path.join(os.getcwd(), results_folder)
if os.path.isdir(output_path):
    shutil.rmtree(output_path)
os.mkdir(output_path)


The training_config.json file consists of the test- train split. With the helper functions in the "training_config.py" we can access the same and use it to load the training or test images and their corresponding labels as required. 

In [12]:
images_root_dir = os.path.join(dataset_root_dir, 'frames')
labels_root_dir = os.path.join(dataset_root_dir, 'frames_txt')

if not (os.path.exists(images_root_dir) and os.path.exists(labels_root_dir)):
    raise FileNotFoundError()
    
config = training_config.get(dataset_root_dir)

if config is not None:
    to_predict_filenames = config['test']
else:
    to_predict_filenames = [os.path.splitext(x)[0] for x in os.listdir(images_root_dir)]

INFO:segmentation.training_config:Loading training config from /home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/dataset/training_config.json


In [13]:
labels = [bee_dataset.read_label_file_globalcoords(os.path.join(labels_root_dir, name + '.txt')) for name in to_predict_filenames]
regions_of_interest = [l[1] for l in labels]

The trained model is then used to give the predictions on the test images that have been considered.

In [14]:
estimator = tf.estimator.Estimator(model_fn=partial(model.build_model,
                                                        num_classes=num_classes,
                                                        data_format=data_format,
                                                        bg_fg_weight=None), model_dir=checkpoint_dir)

predictions = estimator.predict(input_fn=partial(dataset.make_dataset,
                                                     data_generator=partial(bee_dataset.generate_predict,
                                                                            images_root_dir=images_root_dir,
                                                                            filenames=to_predict_filenames,
                                                                            regions_of_interest=regions_of_interest),
                                                     data_format=data_format,
                                                     batch_size=1,
                                                     mode=tf.estimator.ModeKeys.PREDICT))

drawing_functions = bee_dataset.get_object_drawing_functions()

TP_count, FP_count, FN_count, correct_type_count = 0, 0, 0, 0
all_pixel_dist, all_axis_diff = [], []

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01483f4a50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/home/kartik/Desktop/DeepLearningLecture_Schutera/Projects/segmentation/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01483f4a50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


For each input frame, there will be 3 images generated - 

1. Segmentation map of bee bodies and abdomen
2. Comparison of the predictions and label
3. Diagram of the True Positives, False positives and False negatives

A text file with the above information is also generated with the help of the save_error_file function defined earlier. 

In [16]:
for name, prediction, label in zip(to_predict_filenames, predictions, labels):

    input_image = prediction['input_data']
    pred_image = prediction['prediction']

    channels_axis = 0 if data_format == 'channels_first' else -1
    amax = np.argmax(pred_image, axis=channels_axis)

    input_image = np.uint8(np.squeeze(input_image) * 255)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2BGR)

    plot_segmentation_map(input_image, amax,
                              os.path.join(output_path, "{}_seg_map.png".format(name)), num_classes=num_classes)

    predictions_pos = find_positions(amax, min_blob_size_px, max_blob_size_px)
    if len(predictions_pos) == 0:
        continue

    np.savetxt(os.path.join(output_path, "{}_predictions.csv".format(name)), predictions_pos, fmt="%i,%i,%i,%.4f")
    np.savetxt(os.path.join(output_path, "{}_labels.csv".format(name)), label[0], fmt="%i,%i,%i,%.4f")

    plot_positions(input_image, [label[0], predictions_pos], [(0, 250, 255), (0, 0, 255)],
                    os.path.join(output_path, "{}_mixed.png".format(name)),
                    drawing_params=drawing_functions)

    pixel_dist, axis_diff, correct_type, TP_results, FN_results, FP_results \
        = compute_error_metrics(np.array(label[0]), np.array(predictions_pos), dist_min=min_distance_px)

    TP_count += len(TP_results)
    FN_count += len(FN_results)
    FP_count += len(FP_results)
    correct_type_count += correct_type
    all_pixel_dist += pixel_dist
    all_axis_diff += axis_diff

    plot_TP_FN_FP(input_image, TP_results, FN_results, FP_results,
                    os.path.join(output_path, "{}_detail.png".format(name)), drawing_functions)

save_error_file(all_pixel_dist, all_axis_diff, TP_count, FP_count, FN_count, correct_type_count,
                os.path.join(output_path, "average_error_metrics.txt"))